In [2]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm

/home/ipausers/bisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
directory = '/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/'
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']==799]

In [4]:
dir_drop = 'prince_data'


In [ ]:
run_info = folders.copy()
folder_list = list(run_info['folder'])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info['unique_id'] = run_info['Plate'].astype(str) + "_" + run_info['CrossDate'].astype(str)
        line = run_info.loc[run_info['folder']==directory_name]
        id_unique = line['unique_id'].iloc[0]

        path_snap=directory+directory_name
        API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))

        path_zip =f'{os.getenv("TEMP")}{directory_name}.zip'
        path_info =f'{os.getenv("TEMP")}{directory_name}_info.json'
        line.to_json(path_info)
        zip_file(path_snap,path_zip)
        upload(API,path_zip,f'/{dir_drop}/{id_unique}/{directory_name}.zip',chunk_size=256 * 1024 * 1024)
        upload(API,path_info,f'/{dir_drop}/{id_unique}/{directory_name}_info.json',chunk_size=256 * 1024 * 1024)
        os.remove(path_info) 
        os.remove(path_zip)
        pbar.update(1)

transferred:   0%|          | 0/135 [00:00<?, ?it/s]

zipping:   0%|          | 0/158 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/962303402 [00:00<?, ?it/s]

zipping:   0%|          | 0/171 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/950029602 [00:00<?, ?it/s]

...

zipping:   0%|          | 0/171 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/950221941 [00:00<?, ?it/s]

zipping:   0%|          | 0/171 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/950565667 [00:00<?, ?it/s]

zipping:   0%|          | 0/152 [00:00<?, ?it/s]